In [ ]:
spark.sql('show databases').show()

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 100)
spark.sql('use covid_2020_q3')

In [ ]:
### Explore data (1)
spark.sql('select count (*) from demographics_corrected where deceased=1').toPandas()

In [ ]:
### Explore data (2)
df_dbyt=spark.sql('select tenant,deceased, count(distinct personid) cnt from demographics_corrected group by tenant,deceased order by cnt desc ').toPandas()### only 7 tenant have more han 10K COVID patient
df_dbyt.groupby('tenant')['cnt'].sum().sort_values(ascending=False)[:15].plot.bar(x='tenant', y='cnt') 

In [ ]:
### To Create Index tables:
## Case
spark.sql('create table LR_Dec.LR_enc_case1 as select personid,encounterid,hospitalizationstartdate, servicedate, (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) adm_date,dischargedate,dischargedisposition from encounter where personid in (select distinct personid from demographics where deceased=1) and (covid_dx_ind=1 or pos_covid_lab_ind=1 or pos_covid_lab_2wk_prior_ind=1) ')
spark.sql('create table LR_Dec.LR_enc_case2 as select personid,count(distinct encounterid) enc_cnt, max(hospitalizationstartdate) last_hosp, min(servicedate) min_service, min(case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) min_adm_date, max(dischargedate) max_disch,collect_list(dischargedisposition) disc_disps from LR_Dec.LR_enc_case1 group by personid  ')

## Ctrl
spark.sql('create table LR_Dec.LR_enc_ctrl1 as select personid,encounterid,hospitalizationstartdate, servicedate, (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) adm_date,dischargedate,dischargedisposition from encounter where personid in (select distinct personid from demographics where deceased=0) and (covid_dx_ind=1 or pos_covid_lab_ind=1 or pos_covid_lab_2wk_prior_ind=1)  ')
spark.sql('create table LR_Dec.LR_enc_ctrl2 as select personid,count(distinct encounterid) enc_cnt, max(hospitalizationstartdate) last_hosp, min(servicedate) min_service, min(case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) min_adm_date, max(dischargedate) max_disch,collect_list(dischargedisposition) disc_disps from LR_Dec.LR_enc_ctrl1 group by personid ')

## Further cleaning, removing patients discharged to hospice from controls
## and removing patients that their whole history is less than one day, numbers in the cell below
spark.sql("create table LR_Dec.lr_hospice_pt_excl as select distinct personid from LR_Dec.LR_enc_ctrl2 where lower(array_join(disc_disps,'|')) like '%hospice%'") #### 238 pts , exclude
spark.sql('create table LR_Dec.Lr_case_los_excl as select distinct personid from LR_Dec.LR_enc_case2 where datediff(max_disch,min_adm_date)<1')
spark.sql('create table LR_Dec.Lr_ctrl_los_excl as select distinct personid from LR_Dec.LR_enc_ctrl2 where datediff(max_disch,min_adm_date)<1').toPandas()

### Creating the first version of index tables
spark.sql('create table LR_Dec.LR_ctrl_index1 select distinct personid, min_adm_date from LR_Dec.LR_enc_ctrl2 where personid not in (select distinct personid from LR_Dec.lr_hospice_pt_excl) and personid not in (select distinct personid from LR_Dec.Lr_ctrl_los_excl)')
spark.sql('create table LR_Dec.LR_case_index1 select distinct personid, min_adm_date from LR_Dec.LR_enc_case2 where personid not in (select distinct personid from LR_Dec.Lr_case_los_excl)')

In [ ]:
#### Data Review (1)
#spark.sql('select count(*), count(distinct personid), count(distinct encounterid) from LR_Dec.LR_enc_case1').toPandas() ###29100 	20976 	29100
#spark.sql('select count(distinct personid) from LR_Dec.LR_enc_case2 where datediff(max_disch,min_adm_date)<=1').toPandas()
####1410 <1 , 3145 <=1 out of 20976 --- dropped <1  as a primary condition
#spark.sql('select count(distinct personid) from LR_Dec.LR_case_index1 ').toPandas() #### 19,566
#spark.sql('select count(distinct personid) from LR_Dec.LR_enc_ctrl2 where datediff(max_disch,min_adm_date)<1').toPandas()
#### 200397 <1 , 274,452 <=1 out of  469,397 --- dropped <1 as a primary condition
#spark.sql('select count(distinct personid) from LR_Dec.LR_ctrl_index1 ').toPandas()#### 265,408



In [ ]:
### need to get the last day in the encounter table regardless if the encounter was flagged with pos cvd diag/lab or not
spark.sql('create table LR_Dec.lr_case_dis as select personid, max((case when dischargedate="" then (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) else  dischargedate end)) max_dis_date from encounter where personid in (select distinct personid from LR_Dec.lr_case_index1) group by personid')
spark.sql('create table LR_Dec.lr_ctrl_dis as select personid, max((case when dischargedate="" then (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) else  dischargedate end)) max_dis_date from encounter where personid in (select distinct personid from LR_Dec.lr_ctrl_index1) group by personid')
### add this information to the index table
spark.sql('create table LR_Dec.lr_case_index_ttl as select distinct i.*, d.max_dis_date, datediff(d.max_dis_date,i.min_adm_date) ttl from LR_Dec.lr_case_index1 i ,LR_Dec.lr_case_dis d where i.personid= d.personid ')
spark.sql('create table LR_Dec.lr_ctrl_index_ttl as select distinct i.*, d.max_dis_date, datediff(d.max_dis_date,i.min_adm_date) ttl from LR_Dec.lr_ctrl_index1 i ,LR_Dec.lr_ctrl_dis d where i.personid= d.personid ')
### Cleaning data as described in the data review (2) Cell below
spark.sql(' create table LR_Dec.lr_ctrl_index1_ttl as select * from LR_Dec.lr_ctrl_index_ttl where ttl>=1')
spark.sql(' create table LR_Dec.lr_case_index1_ttl as select * from LR_Dec.lr_case_index_ttl where ttl>=1')

#### Creating a list of all encounters after cvd diag
spark.sql('create table LR_Dec.LR_case_After_cvd_encs as select e.*,i.min_adm_date,i.max_dis_date from LR_Dec.lr_case_index1_ttl i left join encounter e on i.personid=e.personid and i.min_adm_date <=(case when e.hospitalizationstartdate="" then e.servicedate else e.hospitalizationstartdate end)')
spark.sql('create table LR_Dec.LR_ctrl_After_cvd_encs as select e.*,i.min_adm_date,i.max_dis_date from LR_Dec.lr_ctrl_index1_ttl i left join encounter e on i.personid=e.personid and i.min_adm_date <=(case when e.hospitalizationstartdate="" then e.servicedate else e.hospitalizationstartdate end)')

#### clean to focus on hospitalizations ( all cause not a must to be CVD labelled)
spark.sql(' create table LR_Dec.LR_case_Aft_cvd_hosp_encs as select *, (case when hospitalizationstartdate="" then servicedate else hospitalizationstartdate end) adm_date,(case when dischargedate="" then NULL else dischargedate end) dis_date from LR_Dec.LR_case_After_cvd_encs where encountertype in ("Emergency","Inpatient","Admitted for Observation","Urgent care encounter")')
spark.sql(' create table LR_Dec.LR_ctrl_Aft_cvd_hosp_encs as select *, (case when hospitalizationstartdate="" then servicedate else hospitalizationstartdate end) adm_date,(case when dischargedate="" then NULL else dischargedate end) dis_date from LR_Dec.LR_ctrl_After_cvd_encs where encountertype in ("Emergency","Inpatient","Admitted for Observation","Urgent care encounter")')

#### Creating the Metadata 
### sorting and ranking hospitalization after the first CVD encounter
spark.sql('create table LR_Dec.LR_case_Aft_cvd_hosp_encs_met as select *, dense_rank() OVER (PARTITION BY personid ORDER BY (to_date(adm_date))) as enc_order, lag(adm_date,1) OVER (PARTITION BY personid ORDER BY (to_date(adm_date))) as prev_adm_date, lag(dis_date,1) OVER (PARTITION BY personid ORDER BY (to_date(adm_date))) as prev_dis_date,datediff(adm_date,nvl(lag(dis_date,1) OVER (PARTITION BY personid ORDER BY (to_date(adm_date))),adm_date) ) time_diff_bet_encs, datediff(dis_date,adm_date) LOS from LR_Dec.LR_case_Aft_cvd_hosp_encs order by personid, adm_date ')
spark.sql('create table LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met as select *, dense_rank() OVER (PARTITION BY personid ORDER BY (to_date(adm_date))) as enc_order, lag(adm_date,1) OVER (PARTITION BY personid ORDER BY (to_date(adm_date))) as prev_adm_date, lag(dis_date,1) OVER (PARTITION BY personid ORDER BY (to_date(adm_date))) as prev_dis_date,datediff(adm_date,nvl(lag(dis_date,1) OVER (PARTITION BY personid ORDER BY (to_date(adm_date))),adm_date) ) time_diff_bet_encs, datediff(dis_date,adm_date) LOS from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs order by personid, adm_date ')
### Finding the LOS of the first encounter, used Max id 2 encounters start at the same time, like pt come to emrgency and move to inpatient
spark.sql('create table LR_Dec.LR_case_LOS_p1 as select personid,min_adm_date,max_dis_date,min(adm_date) min_1rstenc_adm_dt,max(dis_date) max_1rstenc_dis_dt,max(LOS)max_los, collect_list(distinct encountertype) enc_types,collect_list(distinct dischargedisposition) disch_dispos from LR_Dec.LR_case_Aft_cvd_hosp_encs_met where enc_order=1 group by personid,min_adm_date,max_dis_date')
spark.sql('create table LR_Dec.LR_ctrl_LOS_p1 as select personid,min_adm_date,max_dis_date,min(adm_date) min_1rstenc_adm_dt,max(dis_date) max_1rstenc_dis_dt,max(LOS)max_los, collect_list(distinct encountertype) enc_types,collect_list(distinct dischargedisposition) disch_dispos from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met where enc_order=1 group by personid,min_adm_date,max_dis_date')
### create information for first encounter post-covid
spark.sql('create table LR_Dec.LR_post_1rscvd as select distinct personid,max_1rstenc_dis_dt,max_los enc1_los, concat_ws("|",enc_types) enc1_types from LR_Dec.LR_case_LOS_p1 where max_los is not null union select distinct personid,max_1rstenc_dis_dt,max_los enc1_los, concat_ws("|",enc_types) enc1_types from LR_Dec.LR_ctrl_LOS_p1 where max_los is not null')
### Finding the longest hospitalisation per patient regardless if it is the first hospitalization or not
spark.sql('create table LR_Dec.LR_maxLOS_pt as select distinct personid,max(LOS) max_los_pt from LR_Dec.LR_case_Aft_cvd_hosp_encs_met group by personid union select distinct personid,max(LOS) max_los_pt from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met group by personid' )
### other characteristics like rehospitalization and readmission (must be inpatient)
###, number of hospitalization after cvd and number of inpatient hospitalization
spark.sql('create table LR_Dec.LR_Rehosp_pt as select distinct personid from LR_Dec.LR_case_Aft_cvd_hosp_encs_met where enc_order>1 and time_diff_bet_encs>1 union select distinct personid from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met where enc_order>1 and time_diff_bet_encs>1' )
spark.sql('create table LR_Dec.LR_Readm_pt as select distinct personid from LR_Dec.LR_case_Aft_cvd_hosp_encs_met where enc_order>1 and time_diff_bet_encs>1 and los>=1 and encountertype="Inpatient" and datediff(prev_dis_date,prev_adm_date)>=1 union select distinct personid from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met where enc_order>1 and time_diff_bet_encs>1 and los>=1 and encountertype="Inpatient" and datediff(prev_dis_date,prev_adm_date)>=1' )
spark.sql('create table LR_Dec.LR_post_cvdenc_cnts_pt as select distinct personid,max(enc_order) post_cvd_enc_cnt  from LR_Dec.LR_case_Aft_cvd_hosp_encs_met group by personid union select distinct personid , max(enc_order) post_cvd_enc_cnt from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met group by personid' )
spark.sql('create table LR_Dec.LR_post_cvdInpatenc_cnts_pt as select distinct personid,count( distinct enc_order) post_cvd_inp_enc_cnt  from LR_Dec.LR_case_Aft_cvd_hosp_encs_met where lower(encountertype) like "%inpatient%" group by personid union select distinct personid , count( distinct enc_order) post_cvd_inp_enc_cnt from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met where lower(encountertype) like "%inpatient%" group by personid' )
### to flag patients who die on their first encounter
spark.sql('create table LR_Dec.LR_Mort_1rst_enc_pt as select distinct personid from LR_Dec.LR_case_LOS_p1 where lower(concat_ws(",",disch_dispos)) like "%hospice%" or lower(concat_ws(",",disch_dispos)) like "%expire%"')
#### Age information
spark.sql('create table LR_Dec.LR_pt_age as select personid,min(age_at_encounter) pt_age from LR_Dec.LR_case_Aft_cvd_hosp_encs_met group by personid union select personid,min(age_at_encounter) pt_age from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met group by personid') 


In [ ]:
spark.sql('select max(time_diff_bet_encs) from LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met').toPandas()

In [ ]:
### Data Review (2)
#spark.sql('select count(*), count(distinct personid) from LR_Dec.lr_ctrl_index_ttl where ttl<=0').toPandas() ##<0 0 perfect, =0 1718
#spark.sql('select count(*), count(distinct personid) from LR_Dec.lr_case_index_ttl where ttl<=0').toPandas() ##<0 0 perfect, =0 12 ## drop those
#### Review Counts per encounter type
#spark.sql('select encountertype, count(distinct personid) pcnt from LR_Dec.LR_case_After_cvd_encs where encountertype in ("Emergency","Inpatient","Admitted for Observation","Urgent care encounter") group by encountertype order by pcnt desc').toPandas()
###### if I only focus on inpatient we have 19124 case patients
'''encountertype 	pcnt
0 	Inpatient 	19124
1 	Emergency 	2693
2 	Admitted for Observation 	658
3 	Urgent care encounter 	78'''

#spark.sql('select encountertype, count(distinct personid) pcnt from LR_Dec.LR_ctrl_After_cvd_encs where encountertype in ("Emergency","Inpatient","Admitted for Observation","Urgent care encounter") group by encountertype order by pcnt desc').toPandas()
########### if I only focus on inpatient we have 177442 control patients, Emergency 	94466,Admitted for Observation 	38535
'''
encountertype 	pcnt
0 	Inpatient 	177442
1 	Emergency 	94466
2 	Admitted for Observation 	38535
3 	Urgent care encounter 	12049
'''
#spark.sql('select count(*), count (distinct personid) from LR_Dec.lr_ctrl_index1_ttl').toPandas() ###263690
#spark.sql('select count(*), count (distinct personid) from LR_Dec.lr_case_index1_ttl').toPandas() ###19554

spark.sql('select * from LR_Dec.LR_case_Aft_cvd_hosp_encs limit 10').toPandas()


In [ ]:
#### Creating Intubation information

#### to get vent procedures
spark.sql(' create table LR_DEC.lr_case_index1_vent as select distinct p.*,i.min_adm_date,i.max_dis_date from LR_DEC.lr_case_index1_ttl i,procedure p  where i.personid=p.personid and to_date(case when p.servicestartdate="" then i.min_adm_date else p.servicestartdate end )>= to_date(i.min_adm_date) and (procedurecode in ("78582","78598","94002","94003","94660","94004","93.90","96.70","96.71","96.72","47545007","243142003","429487005","45444009") or  procedurecode like "5A_9%" )') 
spark.sql(' create table LR_DEC.lr_ctrl_index1_vent as select distinct p.*,i.min_adm_date,i.max_dis_date from LR_DEC.lr_ctrl_index1_ttl i,procedure p  where i.personid=p.personid and to_date(case when p.servicestartdate="" then i.min_adm_date else p.servicestartdate end )>= to_date(i.min_adm_date) and (procedurecode in ("78582","78598","94002","94003","94660","94004","93.90","96.70","96.71","96.72","47545007","243142003","429487005","45444009") or  procedurecode like "5A_9%" )') 
spark.sql('create table LR_DEC.lr_case_postindex_ventproc as select personid, min(case when servicestartdate="" then NULL else servicestartdate end) min_postindex_ventproc_dt from LR_DEC.lr_case_index1_vent group by personid')
spark.sql('create table LR_DEC.lr_ctrl_postindex_ventproc as select personid, min(case when servicestartdate="" then NULL else servicestartdate end) min_postindex_ventproc_dt from LR_DEC.lr_ctrl_index1_vent group by personid')

#### to get vent labs
spark.sql(' create table LR_DEC.lr_case_index1_vent2 as select distinct r.*,i.min_adm_date,i.max_dis_date from LR_DEC.lr_case_index1_ttl i, result r  where i.personid=r.personid and to_date(case when r.servicedate="" then i.min_adm_date else r.servicedate end )>= to_date(i.min_adm_date) and (r.resultcode in ("47545007","251901004","19834-1","19835-8","19839-0","19840-8","19932-3","19976-0","19994-3","19996-8","20054-3","20055-0","20056-8","20058-4","20063-4","20068-3","20077-4","20079-0","20112-9","20116-0","20124-4","33429-2","33438-3","33446-6","60794-5","76007-4","76222-9","26261000175105"))') 
spark.sql(' create table LR_DEC.lr_ctrl_index1_vent2 as select distinct r.*,i.min_adm_date,i.max_dis_date from LR_DEC.lr_ctrl_index1_ttl i, result r  where i.personid=r.personid and to_date(case when r.servicedate="" then i.min_adm_date else r.servicedate end )>= to_date(i.min_adm_date) and (r.resultcode in ("47545007","251901004","19834-1","19835-8","19839-0","19840-8","19932-3","19976-0","19994-3","19996-8","20054-3","20055-0","20056-8","20058-4","20063-4","20068-3","20077-4","20079-0","20112-9","20116-0","20124-4","33429-2","33438-3","33446-6","60794-5","76007-4","76222-9","26261000175105"))') 
spark.sql('create table LR_DEC.lr_case_postindex_ventlab as select personid, min(case when servicedate="" then NULL else servicedate end) min_postindex_ventlab_dt from LR_DEC.lr_case_index1_vent2 group by personid')
spark.sql('create table LR_DEC.lr_ctrl_postindex_ventlab as select personid, min(case when servicedate="" then NULL else servicedate end) min_postindex_ventlab_dt from LR_DEC.lr_ctrl_index1_vent2 group by personid')

#### Combine vent proc and vent lab and select the minimum date
spark.sql('create table  LR_DEC.lr_case_index1_vent3 as select i.*,p.min_postindex_ventproc_dt,l.min_postindex_ventlab_dt from  LR_DEC.lr_case_index1_ttl i left join LR_DEC.lr_case_postindex_ventproc p on i.personid=p.personid left join LR_DEC.lr_case_postindex_ventlab l on i.personid=l.personid')
spark.sql('create table  LR_DEC.lr_ctrl_index1_vent3 as select i.*,p.min_postindex_ventproc_dt,l.min_postindex_ventlab_dt from  LR_DEC.lr_ctrl_index1_ttl i left join LR_DEC.lr_ctrl_postindex_ventproc p on i.personid=p.personid left join LR_DEC.lr_ctrl_postindex_ventlab l on i.personid=l.personid')

spark.sql('create table LR_DEC.lr_case_index1_ttl_vent as select *, least(min_postindex_ventproc_dt,min_postindex_ventlab_dt) min_postindex_vent_dt from LR_DEC.lr_case_index1_vent3')
spark.sql('create table LR_DEC.lr_ctrl_index1_ttl_vent as select *, least(min_postindex_ventproc_dt,min_postindex_ventlab_dt) min_postindex_vent_dt from LR_DEC.lr_ctrl_index1_vent3')


In [ ]:
### Data Review (3)
#spark.sql('select * from LR_DEC.lr_case_index1_vent where servicestartdate="" ').toPandas() #### 12 record
#spark.sql('select * from LR_DEC.lr_ctrl_index1_vent where servicestartdate="" ').toPandas() #### 41 record
#spark.sql('select b.* from (select * from LR_DEC.lr_case_index1_vent where servicestartdate="")a , encounter b where a.personid=b.personid and a.encounterid=b.encounterid ').toPandas() #### all 12 encounters has servicedate, better than earlier refresh
#spark.sql('select * from (select * from LR_DEC.lr_ctrl_index1_vent where servicestartdate="") a left join encounter b on a.encounterid=b.encounterid').toPandas() #### 12 record
##### as I'm also adding labs, so if missed will convert to nulls and select the min value (will ignore nulls) and then will get the min between lab or proc
#spark.sql('select count(distinct personid) from LR_DEC.lr_case_index1_vent3 where min_postindex_ventproc_dt is not null and min_postindex_ventlab_dt is not null').toPandas() 
### 4351 are in ventlab and not ventproc and 378 with ventproc and not ventlab, and 9041 have both ventlab and ventproc
#spark.sql('select count(distinct personid) from LR_DEC.lr_case_index1_vent3 where to_date(min_postindex_ventproc_dt)= to_date(min_postindex_ventlab_dt) ').toPandas() 
### out of the 9041, 5230 have the same lab and proc date


In [ ]:
#### Create outcomes summary table
spark.sql('create table LR_DEC.LR_cohort_outcomes1 as select * , "1" Mortality_flag from LR_DEC.lr_case_index1_ttl_vent union select * , "0" Mortality_flag from LR_DEC.lr_ctrl_index1_ttl_vent')

### just keep that one and the last one
spark.sql('create table LR_Dec.LR_cohort_outcomes2 as select i.* , rh.personid Rehosp_flag, ra.personid Readm_flag ,m1.personid mort_1enc_flag, ml.max_los_pt, ec.post_cvd_enc_cnt , iec.post_cvd_inp_enc_cnt'
          + ' from LR_Dec.LR_cohort_outcomes1 i left join LR_Dec.LR_Rehosp_pt rh on i.personid=rh.personid '
          + ' left join LR_Dec.LR_Readm_pt ra on i.personid=ra.personid '
          + ' left join LR_Dec.LR_Mort_1rst_enc_pt m1 on i.personid=m1.personid '
          + ' left join LR_Dec.LR_maxLOS_pt ml on i.personid=ml.personid '
          + ' left join LR_Dec.LR_post_cvdenc_cnts_pt ec on i.personid=ec.personid '
          + ' left join LR_Dec.LR_post_cvdInpatenc_cnts_pt iec on i.personid=iec.personid ')

spark.sql('create table LR_Dec.LR_cohort_outcomes3 as select i.* ,enc1.max_1rstenc_dis_dt, enc1.enc1_los, enc1.enc1_types '
          + ' from LR_Dec.LR_cohort_outcomes2 i , LR_Dec.LR_post_1rscvd enc1 where i.personid=enc1.personid ')

#### Summary descriptive table
spark.sql('create table LR_Dec.LR_cohort_outcome_demo as select distinct i.personid,min_adm_date,max_dis_date,Mortality_flag,max_1rstenc_dis_dt,(case when nvl(mort_1enc_flag,0)==0 then "0" else "1" end) mort_enc1_flag,enc1_types,enc1_los,max_los_pt,(case when nvl(min_postindex_vent_dt,0)==0 then "0" else "1" end) vent_flag, min_postindex_vent_dt vent_date,nvl(post_cvd_inp_enc_cnt,0) cvd_inp_encs ,post_cvd_enc_cnt  cvd_encs, (case when nvl(Readm_flag,0)==0 then "0" else "1" end) readm_flag , (case when nvl(Rehosp_flag,0)==0 then "0" else "1" end) rehosp_flag,ttl,a.pt_age,(case when lower(race) like "%racial%" then "Other" else race end) race, (case when lower(gender) like "%unknown%" then "Other" else gender end) gender,ethnicity ,zip_code  from LR_Dec.LR_cohort_outcomes3 i left join LR_Dec.LR_pt_age a on i.personid=a.personid left join demographics d on i.personid=d.personid ')

### For inhospital mortalility,vent, and plos prediction on 1rst covid hospitalization
spark.sql('create table LR_Dec.LR_inhosp_mort_tmp as select distinct personid, min_adm_date,mort_enc1_flag,ttl ttd,vent_flag, nvl(datediff(vent_date,min_adm_date),enc1_los) tti,(case when enc1_los>3 then "1" else "0" end) plos_flag ,enc1_los,pt_age,race,gender,ethnicity,zip_code from lr_dec.LR_cohort_outcome_demo where mort_enc1_flag= Mortality_flag and enc1_los>=1 and to_date(nvl(vent_date,"1900-01-01T16:12:00+00:00")) <= to_date(max_1rstenc_dis_dt)')

In [ ]:
### Review Data (4)
spark.sql('select count(*),count(distinct personid) from LR_Dec.LR_inhosp_mort_tmp ').toPandas() 


In [ ]:
#### Create data tables ----- more details on Data_Extraction_CRWD notebook
### for example Diagnosis
spark.sql('create table LR_dec.LR_cond as select distinct personid,encounterid, (case when effectivedate=="" then asserteddate else effectivedate end) diagdate,codetype, conditioncode, source_encounter_type from condition ' )
spark.sql('create table LR_dec.LR_cond2 as select distinct i.*,c.servicedate ,c.hospitalizationstartdate ,c.dischargedate from LR_dec.LR_cond i left join encounter c  on i.personid=c.personid and i.encounterid=c.encounterid')
spark.sql('create table LR_dec.lr_all_diag1 as select distinct c.personid, codetype, conditioncode,(case when dischargedate="" then (case when hospitalizationstartdate="" then servicedate else  hospitalizationstartdate end) else  dischargedate end) vdate ,(case when diagdate="" then (case when dischargedate="" then servicedate else dischargedate end) else diagdate end) event_date from LR_dec.LR_cond2 c, LR_dec.LR_cohort_outcome_demo i where c.personid=i.personid and '
          + ' datediff((case when diagdate="" then (case when dischargedate="" then servicedate else dischargedate end) else diagdate end) , i.max_1rstenc_dis_dt)<1 ')
### Create data for Inhospital mortality diag
#spark.sql('drop table LR_Dec.LR_inhosp_mort_diag_tmp')
spark.sql('create table LR_Dec.LR_inhosp_mort_diag_tmp as  select distinct d.personid, concat("D_",(case when codetype like ("%ICD%10%") then "ICD10" when codetype like ("%ICD%9%D%") then "ICD9" when codetype like ("%SNOMED%") then "SNOMED" end),"_",conditioncode) covar, to_date(vdate) enc_date from LR_Dec.lr_all_diag1 d, LR_Dec.LR_inhosp_mort_tmp p where d.personid=p.personid and (codetype like ("%ICD%10%") or codetype like ("%ICD%9%D%") or codetype like ("%SNOMED%")) and datediff(d.event_date,p.min_adm_date)<1')

In [ ]:
### Lab, Procedure, and demographic data created in the DataPrep_Dec_lab_proc file
### Proc data created in the DataPrep_Dec_lab_proc file
### demo data created in the DataPrep_Dec_lab_proc file
##### Cleaning up the patient information to create the more clean codes ( to avoid duplicate SNOMED and Loincs)
#spark.sql('create table lr_Dec.LR_inhosp_pred_diag_v1 as select distinct personid ,substring(replace(covar,".",""), 3) new_covar ,enc_date from lr_Dec.LR_inhosp_mort_diag_tmp' ).toPandas()
#spark.sql('create table lr_Dec.LR_inhosp_proc_proc_v1 as select distinct personid ,substring(replace(covar,".",""), 3) new_covar ,enc_date from LR_Dec.LR_inhosp_mort_proc_tmp').toPandas()
#spark.sql('create table lr_Dec.LR_inhosp_pred_result_v1 as select distinct personid ,substring(covar, 3) new_covar ,enc_date  from lr_dec.LR_inhosp_mort_lab_tmp_v4 ').toPandas()
#spark.sql('create table lr_Dec.LR_inhosp_pred_med_v1 as select distinct personid ,substring(covar, 3) new_covar ,enc_date from (select * from lr_dec.LR_inhosp_mort_med_tmp where covar not like "%M_u%" union select * from lr_dec.LR_inhosp_mort_med_cat_tmp) x ').toPandas()
#spark.sql('select distinct personid ,covar new_covar ,enc_date from LR_Dec.LR_inhosp_demo_tmp ').toPandas()


In [ ]:
#### Create Cleaner Data - Remove encs that only have Demographics
spark.sql('create table lr_dec.LR_inhosp_pt_dts as select distinct personid, enc_date from (select * from lr_Dec.LR_inhosp_pred_diag_v1 union select * from  lr_Dec.LR_inhosp_proc_proc_v1 union select * from lr_Dec.LR_inhosp_pred_result_v1 union select * from lr_Dec.LR_inhosp_pred_med_v1) x').toPandas()

In [ ]:
###QC
spark.sql('select count (*), count(distinct personid ) from lr_dec.LR_inhosp_pt_dts ').toPandas()

In [ ]:
### Define pts for held out hospitals
import pickle
pickle.dump(df_pts_hosp[df_pts_hosp['tenant']==87]['personid'].to_list(), open('hosp87.pts', 'wb'), -1)
pickle.dump(df_pts_hosp[df_pts_hosp['tenant']==41]['personid'].to_list(), open('hosp41.pts', 'wb'), -1)

In [ ]:
###Descriptive Analysis

In [ ]:
### Comorbidities
#spark.sql('select * from LR_dec.lr_all_diag1 limit 5').toPandas()
pt_diags=spark.sql('select distinct d.personid pt_sk, trim(upper(d.conditioncode)) diag_code from LR_Dec.lr_all_diag1 d, LR_Dec.LR_inhosp_mort_tmp p where d.personid=p.personid and (codetype like ("%ICD%10%") or codetype like ("%ICD%9%D%")) and datediff(d.event_date,p.min_adm_date)<1 and d. personid in (select distinct personid from lr_dec.LR_inhosp_pt_dts)').toPandas()

In [ ]:
sc.addFile("../../elx_comorb.py")
import elx_comorb
from elx_comorb import add_elx_comorb, comrob_desc

com_df=add_elx_comorb(pt_diags)

In [ ]:
com_df.to_csv("../../pts_comorb.csv", index=False)

In [ ]:
comrob_desc(com_df)

In [ ]:
### other example ethnicity
spark.sql('select ethnicity, count(distinct personid ) cnt_pt from demographics_corrected where personid in (select distinct personid from lr_dec.LR_inhosp_pt_dts) group by ethnicity').toPandas()

In [ ]:
spark.sql('select distinct a.personid, a.gender, a.race, a.ethnicity ,c.pt_age,censusRegion, censusDivision from demographics_corrected a left join tenant b on a.tenant=b.tenantId left join LR_Dec.LR_inhosp_mort_tmp c on a.personid=c.personid where a.personid in (select distinct personid from lr_dec.LR_inhosp_pt_dts)').toPandas().to_csv("../../pts_demo.csv", index=False)


In [ ]:
### Cleaning up database

#spark.sql('drop table LR_Dec.LR_enc_ctrl1 ')
#spark.sql('drop table LR_Dec.LR_enc_case1 ')
#spark.sql('drop table LR_Dec.LR_enc_ctrl2 ')
#spark.sql('drop table LR_Dec.LR_enc_case2 ')
#spark.sql('drop table LR_Dec.Lr_case_los_excl ')
#spark.sql('drop table LR_Dec.Lr_ctrl_los_excl ')
#spark.sql('drop table LR_Dec.lr_hospice_pt_excl ')
#spark.sql('drop table LR_Dec.lr_case_dis')
#spark.sql('drop table LR_Dec.lr_ctrl_dis')
#spark.sql('drop table LR_Dec.LR_case_index1')
#spark.sql('drop table LR_Dec.LR_ctrl_index1')
#spark.sql('drop table LR_Dec.LR_case_index_ttl')
#spark.sql('drop table LR_Dec.lr_ctrl_index_ttl')

#spark.sql('drop table LR_Dec.LR_case_After_cvd_encs')
#spark.sql('drop table LR_Dec.LR_ctrl_After_cvd_encs')
#spark.sql(' drop table LR_DEC.lr_case_index1_vent')
#spark.sql(' drop table LR_DEC.lr_ctrl_index1_vent')
#spark.sql(' drop table LR_DEC.lr_ctrl_index1_vent2 ')
#spark.sql(' drop table LR_DEC.lr_case_index1_vent2 ')
#spark.sql(' drop table LR_DEC.lr_ctrl_index1_vent3 ')
#spark.sql(' drop table LR_DEC.lr_case_index1_vent3 ')
#spark.sql(' drop table LR_DEC.lr_case_postindex_ventlab ')
#spark.sql(' drop table LR_DEC.lr_ctrl_postindex_ventlab ')
#spark.sql(' drop table LR_DEC.lr_case_postindex_ventproc ')
#spark.sql(' drop table LR_DEC.lr_ctrl_postindex_ventproc ')
#spark.sql('drop table LR_Dec.LR_case_LOS_p1 ')
#spark.sql('drop table LR_Dec.LR_ctrl_LOS_p1 ')
#spark.sql('drop table LR_Dec.LR_maxLOS_pt ')
#spark.sql('drop table LR_Dec.LR_Rehosp_pt ')
#spark.sql('drop table LR_Dec.LR_Readm_pt ')
#spark.sql('drop table LR_Dec.LR_post_cvdenc_cnts_pt ')
#spark.sql('drop table LR_Dec.LR_post_cvdInpatenc_cnts_pt') 
#spark.sql('drop table LR_Dec.LR_Mort_1rst_enc_pt') 
#spark.sql('drop table LR_Dec.LR_pt_age')
#spark.sql('drop table LR_Dec.LR_post_1rscvd ')
#spark.sql('drop table LR_Dec.LR_cohort_outcomes1')
#spark.sql('drop table LR_Dec.LR_cohort_outcomes3')


#spark.sql('drop table LR_dec.LR_cond')
#spark.sql('drop table LR_dec.LR_cond2')

#spark.sql('drop table LR_Dec.LR_case_Aft_cvd_hosp_encs_met')
#spark.sql('drop table LR_Dec.LR_ctrl_Aft_cvd_hosp_encs_met')
#spark.sql('drop table lr_case_index1_ttl')
#spark.sql('drop table lr_ctrl_index1_ttl')


